# Credit Scoring with Missing Data Analysis

This notebook demonstrates the analysis of credit scoring data with a focus on handling Missing Not At Random (MNAR) data. We'll compare different methods for handling missing data and evaluate their impact on model performance.

## Table of Contents
1. [Setup and Data Loading](#1.-Setup-and-Data-Loading)
2. [Data Preprocessing](#2.-Data-Preprocessing)
3. [Missing Data Analysis](#3.-Missing-Data-Analysis)
4. [Handling Missing Data](#4.-Handling-Missing-Data)
5. [Model Training](#5.-Model-Training)
6. [Model Evaluation](#6.-Model-Evaluation)
7. [Results Comparison](#7.-Results-Comparison)
8. [Conclusions](#8.-Conclusions)

## 1. Setup and Data Loading

First, let's import the necessary libraries and load our data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Import our custom modules
from src.missing_data_handler import MissingDataHandler
from src.model import CreditScoringModel
from src.evaluation import ModelEvaluator
from src.utils import preprocess_data, plot_missingness, plot_feature_distributions, create_correlation_matrix, split_data

# Set random seed for reproducibility
np.random.seed(42)

# Load the data
df = pd.read_csv('../data/raw/credit_data.csv')
print(f"Dataset shape: {df.shape}")

## 2. Data Preprocessing

Let's examine our data and perform initial preprocessing steps.

In [ ]:
# Display basic information about the dataset
print("Dataset Info:")
df.info()

print("\nSummary Statistics:")
df.describe()

In [ ]:
# Preprocess the data
processed_df = preprocess_data(df)
print(f"Processed dataset shape: {processed_df.shape}")

## 3. Missing Data Analysis

Let's analyze the patterns of missing data in our dataset.

In [ ]:
# Visualize missing data patterns
plot_missingness(df)

# Plot feature distributions
plot_feature_distributions(processed_df)

# Create correlation matrix
create_correlation_matrix(processed_df)

## 4. Handling Missing Data

We'll apply different methods to handle missing data and create multiple versions of our dataset.

In [ ]:
# Initialize missing data handler
handler = MissingDataHandler()

# Apply different missing data handling methods
df_mean = handler.mean_imputation(processed_df.copy(), 'target')
df_heckman = handler.heckman_correction(processed_df.copy(), 'target', 'income')
df_basl = handler.basl_method(processed_df.copy(), 'target')

# Store datasets in a dictionary
datasets = {
    'mean_imputation': df_mean,
    'heckman_correction': df_heckman,
    'basl_method': df_basl
}

## 5. Model Training

Now we'll train models using each version of our dataset.

In [ ]:
# Initialize model and evaluator
model = CreditScoringModel()
evaluator = ModelEvaluator()

# Dictionary to store results
results = {}

# Train and evaluate models for each dataset
for method_name, dataset in datasets.items():
    print(f"\nProcessing {method_name}...")
    
    # Prepare data
    X = dataset.drop('target', axis=1)
    y = dataset['target']
    X_train, X_test, y_train, y_test = split_data(X, y)
    
    # Train model
    model.train(X_train, y_train)
    
    # Make predictions
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Evaluate
    results[method_name] = evaluator.evaluate_model(y_test, y_pred, y_pred_proba)

## 6. Model Evaluation

Let's evaluate the performance of each model.

In [ ]:
# Compare model performances
evaluator.compare_models(results)

# Print detailed results
for method_name, metrics in results.items():
    print(f"\nResults for {method_name}:")
    for metric, value in metrics.items():
        print(f"{metric}: {value:.4f}")

## 7. Results Comparison

Let's analyze the differences between the methods.

In [ ]:
# Create comparison visualizations
metrics_to_compare = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
evaluator.compare_models(results, metrics=metrics_to_compare)

## 8. Conclusions

Based on our analysis:

1. **Method Comparison**:
   - [Fill in observations about which method performed best]
   - [Note any interesting patterns in the results]

2. **Practical Implications**:
   - [Discuss what these results mean for credit scoring]
   - [Note any limitations or areas for future research]

3. **Recommendations**:
   - [Provide specific recommendations based on the results]
   - [Suggest best practices for handling missing data in credit scoring]